In [1]:
from datasets import load_dataset
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import clip

In [2]:
dataset = load_dataset("starvector/text2svg-stack")

In [3]:
train_df_all = dataset["train"].to_pandas()
test_df_all = dataset["test"].to_pandas()

In [4]:
print(len(train_df_all))
print(len(test_df_all))

2169710
5709


In [5]:
def preprocess_svg_df(df, top_n):
    # width, height 필터링
    def extract_svg_dims(svg_str):
        match = re.search(r'<svg[^>]*width="(\d+)"[^>]*height="(\d+)"', svg_str)
        if match:
            return int(match.group(1)), int(match.group(2))
        return None, None

    df["svg_dims"] = df["Svg"].apply(extract_svg_dims)
    df_filtered = df[df["svg_dims"].apply(
        lambda dims: dims[0] is not None and 800 <= dims[0] <= 1000 and 800 <= dims[1] <= 1000
    )].copy()

    # SVG 길이 기준 상위 N개 선택
    df_filtered["svg_length"] = df_filtered["Svg"].apply(len)
    df_sorted = df_filtered.sort_values("svg_length", ascending=False).head(top_n).copy()

    # 텍스트/SVG 추출
    input_texts = df_sorted["caption_llava"].astype(str).tolist()
    target_svgs = df_sorted["Svg"].astype(str).tolist()

    return input_texts, target_svgs, df_sorted

In [6]:
input_train, target_train, df_train = preprocess_svg_df(train_df_all, top_n = 10000)
input_test, target_test, df_test = preprocess_svg_df(test_df_all, top_n = 300)

In [7]:
print(len(df_train))
print(len(df_test))

30
11


양자화(Quantization)

LLaMA 2 7B는 엄청 큰 모델이라 일반적인 GPU (예: 12GB~24GB VRAM)에서는 **풀 정밀도(float32)**나 **반 정밀도(float16)**로는 로드조차 안됨

4bit 양자화를 통해서 : 모델 사이즈를 크게 줄이고 (4배 이상 작게) VRAM 사용량도 줄임 / 성능은 최대한 유지

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"
hf_token = ""

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_4bit_enable_fp32_cpu_offload=True  # CPU 오프로드 허용
)


tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # GPU에 자동 분산 로딩
    quantization_config=quant_config,
    token=hf_token
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PEFT(Parameter-Efficient Fine-Tuning) : 모델 전체를 다 학습하지 않고, 일부만 학습해서 성능을 유지하면서도 학습 비용을 낮추는 방법 / 기존 LLM은 수십억 개 파라미터 → 전체 파라미터 학습은 매우 비쌈 / PEFT는 일부 파라미터만 학습해서 성능은 비슷하게, 자원은 적게 -> 대표적인 PEFT 방식 중 하나가 바로 LoRA


LoRA(Low-Rank Adaptation) : 기존 모델 파라미터를 고정한 채,"작은 추가 레이어만 학습"해서 fine-tuning 하는 방법

In [9]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"▶ Trainable / Total = {trainable:,} / {total:,}")

▶ Trainable / Total = 4,194,304 / 3,504,607,232


In [10]:
from datasets import Dataset

# Hugging Face Dataset 구성
train_dataset = Dataset.from_dict({
    "input": input_train,
    "output": target_train
})

# 프롬프트 포맷
def format_chat_prompt(example):
    return f"""Given the <CAPTION>, generate the corresponding svg code.
Return a response WITH SVG CODE that appropriately alignes with the request, IT SHOULD BE VERY ELABORATE.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

Please ensure that the generated SVG code is well-formed, valid, and strictly adheres to these constraints.
Focus on a clear and concise representation of the input description within the given limitations. Always give the complete SVG code with nothing omitted. Never use an ellipsis.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
</constraints>

<CAPTION>
{example['input']}
</CAPTION>

SVG:{example['output']}"""


train_dataset = train_dataset.map(lambda x: {"text": format_chat_prompt(x)})

# 토크나이즈
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = train_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [12]:
device = torch.device("cuda")

model.to(device)
model.train()

from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import default_data_collator

dataloader = DataLoader(tokenized_dataset, batch_size=2, shuffle=True, collate_fn=default_data_collator)
optimizer = AdamW(model.parameters(), lr=1e-4)

def add_labels(example):
    input_ids = example["input_ids"]
    labels = input_ids.copy()

    # "SVG:" 토큰을 기준으로 시작점 찾기
    try:
        svg_start = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("SVG:"))[-1]
        start_idx = input_ids.index(svg_start)
    except ValueError:
        start_idx = 0  # fallback: 전체 예측

    # 마스킹 처리
    labels[:start_idx + 1] = [-100] * (start_idx + 1)

    example["labels"] = labels
    return example

tokenized_dataset = tokenized_dataset.map(add_labels)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])  # 🔥 여기가 핵심


for epoch in range(5):
    total_loss = 0
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)  # ✅ 수정된 부분

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if step % 50 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} Completed. Avg Loss: {total_loss / len(dataloader):.4f}")

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Epoch 1, Step 0, Loss: 0.8743
Epoch 1 Completed. Avg Loss: 0.8414


In [13]:
import cairosvg
from PIL import Image
import io

In [14]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

In [15]:
def generate_svg(caption):
    model.eval()

    prompt = f"""Given the <CAPTION>, generate the corresponding svg code.
Return a response WITH SVG CODE that appropriately alignes with the request, IT SHOULD BE VERY ELABORATE.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

Please ensure that the generated SVG code is well-formed, valid, and strictly adheres to these constraints.
Focus on a clear and concise representation of the input description within the given limitations. Always give the complete SVG code with nothing omitted. Never use an ellipsis.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
</constraints>

<CAPTION>
{caption}
</CAPTION>

SVG:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=4096,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            top_k=50
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded


In [16]:
def svg_to_image(svg_code):
    png_bytes = cairosvg.svg2png(bytestring=svg_code.encode("utf-8"))
    image = Image.open(io.BytesIO(png_bytes)).convert("RGB")
    return image

In [17]:
def compute_clip_score(caption, svg_code):
    if not svg_code.strip():
        return 0.0

    try:
        image = svg_to_image(svg_code)
    except Exception as e:
        print(f"❌ SVG to image failed: {e}")
        return 0.0

    image_input = preprocess(image).unsqueeze(0).to(device)
    text_input = clip.tokenize([caption]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(text_input)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).item()
        return similarity

In [18]:
def extract_svg_from_output(output: str) -> str:
    # "SVG:" 이후 부분을 추출
    if "SVG:" in output:
        output = output.split("SVG:")[-1].strip()
    # 혹시 이상한 HTML/텍스트가 껴도 <svg부터 자름
    svg_start = output.find("<svg")
    if svg_start != -1:
        return output[svg_start:].strip()
    return ""  # fallback


In [19]:
generated_svg = generate_svg("A black and white image of a crescent moon.")

In [26]:
extracted_svg = extract_svg_from_output(generated_svg)

In [27]:
print(extracted_svg)

<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="none" stroke="black" stroke-width="1" class="svg-image">
<circle cx="12" cy="12" r="10" fill="#fff" class="moon" />
<path d="M14 6.983L12 10.546L10 6.983L12 10.546L14 6.983" stroke="#fff" stroke-width="1.5" />
</svg>
